<a href="https://colab.research.google.com/github/Shivani8780/Matrimonial-Profile/blob/main/matching_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>